In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters---> 43     loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
   # print("t",t)
   # print("shape",(ps[t]).shape)
   # print("targets",targets[t])
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 1038 characters, 46 unique.
----
 rzsgc"mzaG4jW:(tJsF1TFfnigvtA)gJhjW1SvfD.:AuekGdf)C(Fjuj,gij:pcbuWzOdm"S1vopvfxCvJuDd"p (krkd"cwbkC1u(9jef.ycTxGa14j9FO:ur,dF9sihov:cACObh 4kgWDsDw9x4ybh fJw .emsT9s1mld"Wocn..x,ss4WNJdTjSznrex1icmToN 
----
iter 0, loss: 95.716032
----
 sre)trae "aoaalo ami )gs xhi dh, nDio"tuTaey9a)nejpnnehl iSeense  to ce  iu4 ljSie)vC eg jenisnoifjo ateomani"sf ai  ien wm1r )e am pj,e t e smyy"zu9ihh eou fhot"So  stroo  neCDswS hnNr"jz arnh"wg tco 
----
iter 100, loss: 96.443984
----
 nhoogynvpiterdiSohusxudnA iodtem  diw ueFaae  tynten wvnna,oopissams aOt enen e pluk inlhlsu  ee anieri hlsnprimea  aawohgechsienseee aopaa4 enonay , ifcsiyoha dgnsgae 49mxFat g ang uA.li seioehdclean 
----
iter 200, loss: 94.912036
----
 n xppiar vxp9rd maunpnact lp uane bs ecsedOpmiwattoyp Nr  mye4fieNAnstist Theeiop Wic dne kgn oTe af lvgp fsse,miafse so. yfe bx lpbt vmt rep ws sfseTba oTdeu ed yptvsa l1aratgm rhn A tiant dmseawhexe 
----
iter 300, loss: 93.205453
----
 o  

KeyboardInterrupt: 

In [2]:
%whos

Variable        Type        Data/Info
-------------------------------------
Whh             ndarray     100x100: 10000 elems, type `float64`, 80000 bytes
Why             ndarray     46x100: 4600 elems, type `float64`, 36800 bytes
Wxh             ndarray     100x46: 4600 elems, type `float64`, 36800 bytes
bh              ndarray     100x1: 100 elems, type `float64`, 800 bytes
by              ndarray     46x1: 46 elems, type `float64`, 368 bytes
char_to_ix      dict        n=46
chars           list        n=46
dWhh            ndarray     100x100: 10000 elems, type `float64`, 80000 bytes
dWhy            ndarray     46x100: 4600 elems, type `float64`, 36800 bytes
dWxh            ndarray     100x46: 4600 elems, type `float64`, 36800 bytes
data            str         Nominative determinism is<...>es for those professions.
data_size       int         1038
dbh             ndarray     100x1: 100 elems, type `float64`, 800 bytes
dby             ndarray     46x1: 46 elems, type `float64`, 368 byt

In [3]:
data

'Nominative determinism is the hypothesis that people are drawn to professions that fit their name. The term was first used in the magazine New Scientist in 1994, after its humorous Feedback column mentioned a book on polar explorations by Daniel Snowman and an article on urology by researchers named Splatt and Weedon. The hypothesis had been suggested by psychologist Carl Jung, citing as an example Sigmund Freud (German for "joy"), who studied pleasure. A few recent empirical studies have indicated that certain professions are disproportionately represented by people with appropriate surnames, though the methods of these studies have been challenged. One explanation for nominative determinism is the theory of implicit egotism, which states that humans have an unconscious preference for things they associate with themselves. An alternative explanation is genetic: an ancestor might have been named Smith or Taylor according to their occupation, and the genes they passed down might correl

In [ ]:
k
k



In [ ]:
34


In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input_en_de.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters---> 43     loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
   # print("t",t)
   # print("shape",(ps[t]).shape)
   # print("targets",targets[t])
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data[0:282]) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+282:p+seq_length+282]]

  # sample from the model now and then
  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 612 characters, 45 unique.
----
 Pöy,rbdslnäSiLwäohfggPü dSuatnvyjw.ffäIöiOnLnHGSccnb'rhAyorCWkihsäGäsyC'rzd.SbgcqmblOnO,tLWWwsfiäSjPSGr.ükEdjyBgIdbnnPCRjvhBrRywmjw,EwaHql'Aü'BsOEüLlüsscSRykehAafPönLWnbvz ze'fuCvRvjfebbcPGoBIEOeuälfW 
----
iter 0, loss: 95.166564
----
 iiüeb. cakhBroeanagt eo if z  hi.t nach atedhm ,äaathesH oh c  Huneceileceznaneäane at k  . eh  g.wenodtedr t .ecemct .eüthBatm tm dane aBewhn.ugisHicm t   .k.eefhi.eftäd.chihhchBeR.Er  leuraseiG ceic 
----
iter 1000, loss: 63.742458
----
 gceitklh lt kakürnsnet hihhüieide  rtelhüAef gufechrda frifewes.er faGh dhift eidenLaibhshisHis inewu e.aln.al rt n c.k.tnewuaeftt  Wioedascec m ataeAem ntE gOfwfüIInof dt  ifnet necer,eOr f .räO nech 
----
iter 2000, loss: 25.679307
----
  du cet  iaezr eiteihech dhihiii.eü rtae Hi    t dhaott nth.säenhih .aeO naneienwr eicenh hm taneihihich danh .t  enhihilHiodtfezchi.ttenst ntnemch atakssenf it dane onnid mtaaauter eh hnnd  ezueidule 
----
iter 3000, loss: 9.975359
----
 le

In [6]:
data[282:290]

'Ich nahm'